# Data assistants
Objetivo: criar um assistant com aplicação em análise de dados.

In [1]:
# Bibliotecas

import csv
import random
from faker import Faker # gera dados fictícios
from datetime import datetime, timedelta
import pandas as pd
from dotenv import load_dotenv
import os
import openai

In [2]:
# Inicializando a biblioteca Faker

fake = Faker()

# Generate csv

In [3]:
# Função para gerar dados fictícios
def generate_data():
    invoice_id = fake.uuid4()[:8]
    branch = random.choice(["A", "B", "C"])
    city = random.choice(["City A", "City B", "City C"])
    customer_type = random.choice(["Member", "Normal"])
    gender = random.choice(["Male", "Female"])
    product_line = random.choice([
        "Health and beauty",
        "Electronic accessories",
        "Home and lifestyle",
        "Sports and travel",
        "Food and beverages",
        "Fashion accessories",
    ])
    unit_price = round(random.uniform(10, 100), 2)
    quantity = random.randint(1, 10)
    tax = round((unit_price * quantity) * 0.05, 2)
    total = round((unit_price * quantity) + tax, 2)
    date = fake.date_between(start_date="-1y", end_date="today").strftime("%Y-%m-%d")
    time = fake.time(pattern="%H:%M:%S")
    payment = random.choice(["Cash", "Credit card", "Ewallet"])
    cogs = round(unit_price * quantity, 2)
    gross_margin_percentage = 4.76  # Valor fixo conforme especificação
    gross_income = tax  # Taxa de 5% é o lucro bruto
    rating = round(random.uniform(4, 10), 1)

    return [
        invoice_id,
        branch,
        city,
        customer_type,
        gender,
        product_line,
        unit_price,
        quantity,
        tax,
        total,
        date,
        time,
        payment,
        cogs,
        gross_margin_percentage,
        gross_income,
        rating,
    ]

# Cabeçalho do arquivo CSV
header = [
    "Invoice ID",
    "Branch",
    "City",
    "Customer type",
    "Gender",
    "Product line",
    "Unit price",
    "Quantity",
    "Tax 5%",
    "Total",
    "Date",
    "Time",
    "Payment",
    "cogs",
    "gross margin percentage",
    "gross income",
    "Rating",
]

In [4]:
# Gerar e salvar os dados em um arquivo CSV
with open("files/sales_data.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(header)

    for _ in range(300):
        writer.writerow(generate_data())

print("Arquivo 'sales_data.csv' gerado com sucesso!")

Arquivo 'sales_data.csv' gerado com sucesso!


## Verificando os dados gerados

In [5]:
data = pd.read_csv('files/sales_data.csv')
data.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,891c16f1,C,City A,Member,Male,Fashion accessories,20.53,6,6.16,129.34,2024-07-12,23:53:37,Ewallet,123.18,4.76,6.16,8.0
1,721920e0,C,City B,Normal,Female,Health and beauty,92.91,1,4.65,97.56,2025-02-26,01:09:16,Credit card,92.91,4.76,4.65,8.4
2,4e14f726,C,City B,Member,Male,Electronic accessories,35.93,5,8.98,188.63,2025-03-22,16:51:41,Credit card,179.65,4.76,8.98,6.0
3,f139037f,B,City B,Normal,Male,Food and beverages,29.77,9,13.40,281.33,2024-06-03,02:32:08,Credit card,267.93,4.76,13.40,4.3
4,d6225cfa,B,City B,Member,Male,Food and beverages,57.04,4,11.41,239.57,2024-07-08,03:39:21,Ewallet,228.16,4.76,11.41,4.9


# OpenAI

In [6]:
# Acessando a API

load_dotenv()
secret_key = os.getenv('OPEN_API_KEY')

In [7]:
# Instanciando 

client = openai.Client(api_key=secret_key)

In [8]:
file = client.files.create(
    file=open('files/sales_data.csv', 'rb'),
    purpose='assistants'
)

In [9]:
print(file.id)

file-8kLvTch5wjqj9StW7oxxA3


In [10]:
assistant = client.beta.assistants.create(
    name="Analista de Dados",
    instructions="Você é um analista que analisa dados da área de vendas",
    tools=[{"type":"code_interpreter"}],
    tool_resources={"code_interpreter":{"file_ids":[file.id]}},
    model="gpt-4o"
)

In [11]:
#pergunta = "Qual o rating médio das vendas do supermercado"
pergunta = "Gere um gráfico de barras com o percentual de vendas por linha de produto"

In [12]:
# Criação da Thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=pergunta
)

In [13]:
# Executa a thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Nome de usuário premium"
)

In [14]:
# Aguarda a thread rodar
import time
while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

In [15]:
# Verifica a resposta
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print(f"Erro {run.status}")

SyncCursorPage[Message](data=[Message(id='msg_IQwNatw1l8fzEOtwLmTg1iUx', assistant_id='asst_9qKO9eZ0qY7aTNx6VOloQren', attachments=[], completed_at=None, content=[ImageFileContentBlock(image_file=ImageFile(file_id='file-TLuhcYcm4wuk3yXucscUmV', detail=None), type='image_file'), TextContentBlock(text=Text(annotations=[], value='Aqui está o gráfico de barras que mostra o percentual de vendas por linha de produto. Cada barra representa a contribuição percentual de cada linha de produto para o total de vendas. Se precisar de mais alguma coisa ou outras análises, é só avisar!'), type='text')], created_at=1745694856, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_uE7CEEcDrT21nI3YBBFVTXLD', status=None, thread_id='thread_5URg0di7GWGHUTBEGEhy5cdg'), Message(id='msg_uOz3VXYgomz8auiZkWYTeCOW', assistant_id='asst_9qKO9eZ0qY7aTNx6VOloQren', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value=

In [16]:
# mensagens.data[0].content[0].text.value

In [17]:
# Analisando os passos do modelo
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [18]:
for step in run_steps.data[::-1]:
    print(f"\n====Step {step.step_details.type}")
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("=" *10)
            print(tool_call.code_interpreter.input)
            print("="*10)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        if message.content[0].type == "text":
            print(message.content[0].text.value)
        if message.content[0].type == "image_file":
            file_id = message.content[0].image_file.file_id
            image_data = client.files.content(file_id)
            with open(f"files/{file.id}.png", "wb") as f:
                f.write(image_data.read())
                print(f"Imagem {file_id} salva")
        
        # print(message.content[0].text.value)


====Step message_creation
Para gerar um gráfico de barras com o percentual de vendas por linha de produto, vamos primeiro examinar o conteúdo do arquivo que você carregou. Isso nos ajudará a entender a estrutura dos dados e a identificar as colunas relevantes. Vou começar lendo o arquivo.

====Step tool_calls
import pandas as pd

# Ler o arquivo para verificar o conteúdo
file_path = '/mnt/data/file-8kLvTch5wjqj9StW7oxxA3'
data = pd.read_csv(file_path)

# Mostrar as primeiras linhas do dataframe
data.head()

====Step message_creation
Os dados contêm várias colunas, das quais as relevantes para nossa análise são:

- `Product line`: A linha de produto da venda.
- `Total`: O valor total de cada transação.

Vamos agora calcular o percentual das vendas totais para cada linha de produto e exibir essa informação em um gráfico de barras.

====Step tool_calls
import matplotlib.pyplot as plt

# Calcular o total de vendas por linha de produto
total_sales_by_product_line = data.groupby('Product li

In [19]:
# Verificar output na pasta 'files'